In [8]:
# 필요한 라이브러리 로드
import os
from data_loading import load_images_and_annotations
from data_processing import convert_annotations
from prepare_dataset import prepare_dataset, create_yaml

In [9]:
# 데이터 경로 설정
path_to_training_images = r'C:\Users\dhshs\Documents\해양 침적 쓰레기 이미지\data\train\images'
path_to_training_annotations = r'C:\Users\dhshs\Documents\해양 침적 쓰레기 이미지\data\train\bbox'
path_to_validation_images = r'C:\Users\dhshs\Documents\해양 침적 쓰레기 이미지\data\validation\images'
path_to_validation_annotations = r'C:\Users\dhshs\Documents\해양 침적 쓰레기 이미지\data\validation\bbox'
path_to_test_images = r'C:\Users\dhshs\Documents\해양 침적 쓰레기 이미지\data\test\images'
path_to_test_annotations = r'C:\Users\dhshs\Documents\해양 침적 쓰레기 이미지\data\test\bbox'

# 데이터 로드 및 전처리
print("Loading and processing training data...")
train_images, train_annotations = load_images_and_annotations(path_to_training_images, path_to_training_annotations)
print("Loading and processing validation data...")
val_images, val_annotations = load_images_and_annotations(path_to_validation_images, path_to_validation_annotations)
print("Loading and processing test data...")
test_images, test_annotations = load_images_and_annotations(path_to_test_images, path_to_test_annotations)

all_annotations = train_annotations + val_annotations + test_annotations
print("Converting annotations...")
converted_annotations = convert_annotations(all_annotations)

# 데이터셋 준비
output_dir = r'C:\Users\dhshs\Documents\해양 침적 쓰레기 이미지\YOLOv8_dataset'
print("Preparing dataset...")
prepare_dataset(train_images + val_images + test_images, converted_annotations, output_dir)

# 클래스 레이블 정의
classes = ['타이어', '통발류', '어망류', '나무', '로프']
create_yaml(output_dir, classes)


Loading and processing training data...
Loading images and annotations from C:\Users\dhshs\Documents\해양 침적 쓰레기 이미지\data\train\images and C:\Users\dhshs\Documents\해양 침적 쓰레기 이미지\data\train\bbox
Loaded 8421 images and 8421 annotations.
Loading and processing validation data...
Loading images and annotations from C:\Users\dhshs\Documents\해양 침적 쓰레기 이미지\data\validation\images and C:\Users\dhshs\Documents\해양 침적 쓰레기 이미지\data\validation\bbox
Loaded 2406 images and 2406 annotations.
Loading and processing test data...
Loading images and annotations from C:\Users\dhshs\Documents\해양 침적 쓰레기 이미지\data\test\images and C:\Users\dhshs\Documents\해양 침적 쓰레기 이미지\data\test\bbox
Loaded 1203 images and 1203 annotations.
Converting annotations...
Converting annotations to new class labels...
Converted 12030 annotations.
Preparing dataset...
Dataset prepared with 9624 training and 2406 validation images.
YAML file created at C:\Users\dhshs\Documents\해양 침적 쓰레기 이미지\YOLOv8_dataset\data.yaml


In [10]:
# YOLOv8 설치
# !pip install ultralytics


In [11]:
path_to_training_images = r'C:\Users\dhshs\Documents\해양 침적 쓰레기 이미지\data\train\images'
path_to_training_annotations = r'C:\Users\dhshs\Documents\해양 침적 쓰레기 이미지\data\train\bbox'
path_to_validation_images = r'C:\Users\dhshs\Documents\해양 침적 쓰레기 이미지\data\validation\images'
path_to_validation_annotations = r'C:\Users\dhshs\Documents\해양 침적 쓰레기 이미지\data\validation\bbox'
path_to_test_images = r'C:\Users\dhshs\Documents\해양 침적 쓰레기 이미지\data\test\images'
path_to_test_annotations = r'C:\Users\dhshs\Documents\해양 침적 쓰레기 이미지\data\test\bbox'

In [13]:
import os
import matplotlib.pyplot as plt
from ultralytics import YOLO

# output_dir 변수 정의 확인
output_dir = r'C:\Users\dhshs\Documents\해양 침적 쓰레기 이미지\YOLOv8_dataset'
print(f"Output directory is set to: {output_dir}")

# 모델 가중치를 저장할 디렉토리 생성
weights_dir = os.path.join(output_dir, 'weights')
os.makedirs(weights_dir, exist_ok=True)
print(f"Model weights will be saved to: {weights_dir}")

# YOLOv8 학습 실행
print("Starting YOLOv8 training...")
model = YOLO('yolov8n.pt')

# 성능 지표 저장용 리스트 초기화
epoch_metrics = []

# Early Stopping을 적용하고 학습 모니터링
results = model.train(
    data=os.path.join(output_dir, 'data.yaml'),
    epochs=50,
    imgsz=256,
    batch=32,
    name="yolo_underwater_trash",
    patience=5,  # Early stopping을 위해 5 epochs 동안 성능 향상이 없으면 중지
    device='cpu'  # CPU로 학습 강제
)

# 매 에포크마다 가중치 저장 및 성능 평가
for epoch in range(50):
    weight_path = os.path.join(weights_dir, f'epoch_{epoch + 1}.pt')
    model.save(weight_path)
    print(f"Weights saved to {weight_path} after epoch {epoch + 1}")

    # 모델 평가
    results = model.val(data=os.path.join(output_dir, 'data.yaml'), split='test')
    metrics = results.metrics
    epoch_metrics.append(metrics)

# 성능 지표 시각화
epochs = list(range(1, 51))
precision = [metrics['precision'] for metrics in epoch_metrics]
recall = [metrics['recall'] for metrics in epoch_metrics]
mAP_50 = [metrics['map50'] for metrics in epoch_metrics]
mAP_95 = [metrics['map'] for metrics in epoch_metrics]

plt.figure(figsize=(12, 8))
plt.plot(epochs, precision, label='Precision')
plt.plot(epochs, recall, label='Recall')
plt.plot(epochs, mAP_50, label='mAP@0.5')
plt.plot(epochs, mAP_95, label='mAP@0.5:0.95')
plt.xlabel('Epoch')
plt.ylabel('Metrics')
plt.title('Model Performance on Test Dataset')
plt.legend()
plt.grid(True)
plt.show()


Output directory is set to: C:\Users\dhshs\Documents\해양 침적 쓰레기 이미지\YOLOv8_dataset
Model weights will be saved to: C:\Users\dhshs\Documents\해양 침적 쓰레기 이미지\YOLOv8_dataset\weights
Starting YOLOv8 training...
Ultralytics YOLOv8.2.28  Python-3.11.7 torch-2.3.1+cpu CPU (12th Gen Intel Core(TM) i5-12450H)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=C:\Users\dhshs\Documents\   \YOLOv8_dataset\data.yaml, epochs=50, time=None, patience=5, batch=32, imgsz=256, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=yolo_underwater_trash16, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None,

train: Scanning C:\Users\dhshs\Documents\해양 침적 쓰레기 이미지\YOLOv8_dataset\train\labels.cache... 0 images, 9624 backgrounds, 0 corrupt: 100%|██████████| 9624/9624 [00:00<?, ?it/s]

WARNING  No labels found in C:\Users\dhshs\Documents\   \YOLOv8_dataset\train\labels.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.



val: Scanning C:\Users\dhshs\Documents\해양 침적 쓰레기 이미지\YOLOv8_dataset\val\labels.cache... 0 images, 2406 backgrounds, 0 corrupt: 100%|██████████| 2406/2406 [00:00<?, ?it/s]

WARNING  No labels found in C:\Users\dhshs\Documents\   \YOLOv8_dataset\val\labels.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.
Plotting labels to c:\Users\dhshs\Documents\CCTV\ultralytics\runs\detect\yolo_underwater_trash16\labels.jpg... 
zero-size array to reduction operation maximum which has no identity
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001111, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)


TensorBoard: model graph visualization added 
Image sizes 256 train, 256 val
Using 0 dataloader workers
Logging results to c:\Users\dhshs\Documents\CCTV\ultralytics\runs\detect\yolo_underwater_trash16
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/50         0G          0      41.28          0          0        256:   1%|          | 3/301 [00:10<17:04,  3.44s/it]

In [ ]:
import os
from ultralytics import YOLO

# 특정 에포크 번호 설정
epoch_number = 1  # 평가할 에포크 번호를 설정

# 저장된 가중치 파일 경로 설정
weight_path = os.path.join(weights_dir, f'epoch_{epoch_number}.pt')

# YOLOv8 모델 로드
model = YOLO(weight_path)

# 테스트 데이터셋을 사용하여 모델 평가
results = model.val(data=os.path.join(output_dir, 'data.yaml'), split='test')

# 평가 결과 출력
print(f"Evaluation results for epoch {epoch_number}:")
print(f"Precision: {results.metrics['precision']}")
print(f"Recall: {results.metrics['recall']}")
print(f"mAP@0.5: {results.metrics['map50']}")
print(f"mAP@0.5:0.95: {results.metrics['map']}")
